In [3]:
!pip install flask-ngrok

In [4]:
from flask_ngrok import run_with_ngrok
from flask import Flask

In [5]:
# MNIST 모델 TF 2.0으로 Training 후 save (tf_mnist_train.py)

import tensorflow as tf
import numpy as np

#학습 데이터 load
((train_data, train_label), (eval_data, eval_label)) = tf.keras.datasets.mnist.load_data()

#data를 정규화하여 28x28로 reshape
train_data = train_data/np.float32(255)
train_data = train_data.reshape(60000, 28, 28, 1)
train_data.shape

eval_data = eval_data/np.float32(255)
eval_data = eval_data.reshape(10000, 28, 28, 1)
eval_data.shape


11493376/11490434 [==============================] - 0s 0us/step


(10000, 28, 28, 1)

In [6]:
from tensorflow.keras import models

#cnn으로 모델 생성
model =models.Sequential()
model.add(tf.keras.layers.Conv2D(32, (5,5), padding='same', activation='relu', input_shape=(28,28,1)))
model.add(tf.keras.layers.MaxPooling2D((2,2)))
model.add(tf.keras.layers.Conv2D(64, (5,5), activation='relu'))
model.add(tf.keras.layers.MaxPooling2D((2,2)))
model.add(tf.keras.layers.Conv2D(64, (5,5), activation='relu'))
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(64, activation='relu'))
model.add(tf.keras.layers.Dense(10, activation='softmax'))
model.summary()

#graph를 생성하고 training
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(train_data, train_label, epochs=5)


test_loss, test_acc = model.evaluate(eval_data, eval_label, verbose=2)
test_acc

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 28, 28, 32)        832       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 14, 14, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 10, 10, 64)        51264     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 1, 1, 64)          102464    
_________________________________________________________________
flatten (Flatten)            (None, 64)                0         
_________________________________________________________________
dense (Dense)                (None, 64)                4

0.9890000224113464

In [7]:
import tensorflow as tf

#save the model.
model_dir = "/content/drive/My Drive/Flask/tmp/tfkeras_mnist"
tf.keras.models.save_model(model, model_dir,save_format="tf")

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: /content/drive/My Drive/Flask/tmp/tfkeras_mnist/assets


python으로 위 코드를 실행하면 모델이 지정된 곳에 저장된다.
해당 모델을 불러서 제대로 예측하는지 확인해보면, 99% 정확도로 예측이 더 정확해야 한다.

In [14]:
import tensorflow as tf
import numpy as np

# eval data 불러오고
((train_data, train_label), (eval_data, eval_label)) = tf.keras.datasets.mnist.load_data()
eval_data = eval_data/np.float32(255)
eval_data = eval_data.reshape(10000, 28, 28, 1)

# 저장한 모델 불러 온뒤
model_dir = "/content/drive/My Drive/Flask/tmp/tfkeras_mnist"
new_model = tf.keras.models.load_model(model_dir)
new_model.summary()

# 그래프를 형성하고,
new_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# 임의의 위치에 있는 MNIST 숫자를 하나 읽어서 예측
random_idx = np.random.choice(eval_data.shape[0])
test_data = eval_data[random_idx].reshape(1, 28, 28, 1)
res = new_model.predict(test_data)

# 제대로 학습되었는지 확인
print ("predict: {}, original: {}".format(np.argmax(res), eval_label[random_idx]))

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 28, 28, 32)        832       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 14, 14, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 10, 10, 64)        51264     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 1, 1, 64)          102464    
_________________________________________________________________
flatten (Flatten)            (None, 64)                0         
_________________________________________________________________
dense (Dense)                (None, 64)                4

모델을 다 생성했으니 Flask 이용하여 웹으로 서빙

In [ ]:
#app.py
!pip install flask_restful
from flask import Flask, render_template
import flask_restful
import tensorflow as tf
import numpy as np

# 데이터를 읽어들이고
((train_data, train_label), (eval_data, eval_label)) = tf.keras.datasets.mnist.load_data()
eval_data = eval_data/np.float32(255)
eval_data = eval_data.reshape(10000, 28, 28, 1)

# 저장해 두었던 모델을 읽어들인 후
model_dir = "/content/drive/My Drive/Flask/tmp/tfkeras_mnist"
new_model = tf.keras.models.load_model(model_dir)
new_model.summary()

#그래프를 생성하고
new_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
# Flask Restful API로 읽어들일 APP을 지정.
app = Flask(__name__)
api = flask_restful.Api(app)
# Flask가 사용할 리소스는 Test 클래스. 
# get 함수가 HTTP Get으로 결과를 읽어들임
class Test(flask_restful.Resource):
    def get(self):
        random_idx = np.random.choice(eval_data.shape[0])
        random_idx
        test_data = eval_data[random_idx].reshape(1, 28, 28, 1)
        res = new_model.predict(test_data)

        return {
            'MNIST predict': np.argmax(res).tolist(),
            'MNIST answer': eval_label[random_idx].tolist()
         }

# Test 클래스를 리소스로 추가. 두번째 인자는 파일의 위치. 
# 우리는 ~/venv/tf_mnist 현재 디렉토리에서 읽을 것이므로 '/'     
api.add_resource(Test, '/')
# 사용하는 포트는 5000번
run_with_ngrok(app)
app.run()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 28, 28, 32)        832       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 14, 14, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 10, 10, 64)        51264     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 1, 1, 64)          102464    
_________________________________________________________________
flatten (Flatten)            (None, 64)                0         
_________________________________________________________________
dense (Dense)                (None, 64)                4

 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://2aab311b57de.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [24/Jun/2020 06:06:15] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [24/Jun/2020 06:06:16] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [24/Jun/2020 06:07:41] "GET / HTTP/1.1" 200 -


In [ ]:
#https://blog.naver.com/dsz08082/221800287035